## Deep Learning
## Assignment 2
<hr />
Previously in 1_notmnist.ipynb, we created a pickle with formatted datasets for training, development and testing on the notMNIST dataset.

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them 
# before procedding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
import os
import sys
from six.moves import cPickle as pickle
from six.moves import range

#### First reload the data we generated in 1_notmnist.ipynb

In [2]:
data_path = '/Users/heany/code/AI-For-NLP/dataset'
pickle_file = 'notMNIST.pickle'

with open(os.path.join(data_path, pickle_file),'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save # hint to help gc free up memory
    
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
len(train_dataset[0])

28

In [4]:
train_dataset[0][:2]

array([[-0.5       , -0.5       , -0.49607843, -0.5       , -0.40588236,
        -0.26078433, -0.30784315, -0.29607844, -0.27254903, -0.27254903,
        -0.27254903, -0.27254903, -0.27254903, -0.27254903, -0.27254903,
        -0.27254903, -0.27254903, -0.27254903, -0.27254903, -0.25686276,
        -0.26078433, -0.2647059 , -0.2882353 , -0.31176472, -0.30392158,
        -0.29215688, -0.30784315, -0.4137255 ],
       [-0.5       , -0.5       , -0.5       , -0.4490196 , -0.22941177,
        -0.4019608 , -0.20980392, -0.21764706, -0.44117647, -0.4372549 ,
        -0.44117647, -0.44117647, -0.44117647, -0.44117647, -0.44117647,
        -0.44117647, -0.44117647, -0.4372549 , -0.44117647, -0.2882353 ,
        -0.35490197, -0.20588236, -0.06078431, -0.20196079, -0.28039217,
        -0.38627452, -0.4254902 , -0.26078433]], dtype=float32)

In [5]:
train_labels[:10]

array([4, 2, 4, 3, 2, 1, 3, 9, 8, 0], dtype=int32)

#### Reformat into a shape that's more adapted to the models we're going to train:
<hr />

+ data as flat matrix
+ labels as float 1-hot encodings.

In [6]:
image_size = 28
num_labes = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size* image_size)).astype(np.float32) # shape(-1, 784)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labes) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)    
    

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [7]:
train_labels.shape

(200000, 10)

In [8]:
train_labels[0]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32)

In [9]:
train_dataset[0].shape

(784,)

In [10]:
train_dataset[0]

array([-0.5       , -0.5       , -0.49607843, -0.5       , -0.40588236,
       -0.26078433, -0.30784315, -0.29607844, -0.27254903, -0.27254903,
       -0.27254903, -0.27254903, -0.27254903, -0.27254903, -0.27254903,
       -0.27254903, -0.27254903, -0.27254903, -0.27254903, -0.25686276,
       -0.26078433, -0.2647059 , -0.2882353 , -0.31176472, -0.30392158,
       -0.29215688, -0.30784315, -0.4137255 , -0.5       , -0.5       ,
       -0.5       , -0.4490196 , -0.22941177, -0.4019608 , -0.20980392,
       -0.21764706, -0.44117647, -0.4372549 , -0.44117647, -0.44117647,
       -0.44117647, -0.44117647, -0.44117647, -0.44117647, -0.44117647,
       -0.4372549 , -0.44117647, -0.2882353 , -0.35490197, -0.20588236,
       -0.06078431, -0.20196079, -0.28039217, -0.38627452, -0.4254902 ,
       -0.26078433, -0.5       , -0.49607843, -0.5       , -0.34705883,
       -0.37843138, -0.42941177, -0.2882353 , -0.35490197, -0.5       ,
       -0.5       , -0.5       , -0.5       , -0.5       , -0.5 

We're first going to train a multinomial <span style="color:red">logistic regression using simple gradient descent</span>.

TensorFlow works like this:

+ First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:
    ```
    with graph.as_default():
        ...
    ```
+ Then you can run the operations on this graph as many times as you want by calling session.run(), providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:
    ```
    with tf.Session(graph=graph) as session:
        ...
    ```

#### Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [11]:
# With gradient descent training, even this much data is prohibitive
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    # Input data.
    # Load the trianing, validation and test data into constants that are attached to the graph
    tf_train_dataset = tf.constant(train_dataset[:train_subset,:])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # These are the parameters that we are going to be training. 
    # The weight matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labes])
    )
    biases = tf.Variable(tf.zeros([num_labes]))
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in Tensorflow, because
    # it's very common, and it can be optimized). We take the average of this 
    # cross-entropy across all trianing examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels = tf_train_labels, logits = logits)
    )
    
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.relu(logits)
    valid_prediction = tf.nn.relu(
        tf.matmul(tf_valid_dataset, weights) + biases
    )
    test_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, weights)+ biases )
    

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
test_prediction.shape

TensorShape([Dimension(10000), Dimension(10)])

#### Let's run this computation and iterate:

In [13]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions,1) == np.argmax(labels,1)) / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as 
    # we described in the graph: random weights for the matrix, zeros for the biases.
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if step % 100 == 0:
            print('Loss at step %d: %f'% (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]
            ))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: % .1f%%' % accuracy(test_prediction.eval(),test_labels))


Initialized
Loss at step 0: 13.572025
Training accuracy: 15.6%
Validation accuracy: 18.4%
Loss at step 100: 2.332918
Training accuracy: 71.3%
Validation accuracy: 70.1%
Loss at step 200: 1.877773
Training accuracy: 74.6%
Validation accuracy: 72.6%
Loss at step 300: 1.630909
Training accuracy: 75.8%
Validation accuracy: 73.3%
Loss at step 400: 1.465596
Training accuracy: 76.6%
Validation accuracy: 74.0%
Loss at step 500: 1.342618
Training accuracy: 77.3%
Validation accuracy: 74.3%
Loss at step 600: 1.245083
Training accuracy: 77.9%
Validation accuracy: 74.5%
Loss at step 700: 1.164709
Training accuracy: 78.4%
Validation accuracy: 74.5%
Loss at step 800: 1.097068
Training accuracy: 78.8%
Validation accuracy: 74.6%
Test accuracy:  82.5%


#### Let's now switch to stochastic gradient descent training instead, which is much faster.

<hr />
The graph will be similar, except that instead of holding all the training data into a constant node, we create a Placeholder node which will be fed actual data at every call of session.run().

In [14]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed 
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labes))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labes])
    )
    biases = tf.Variable(tf.zeros([num_labes]))
    
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels = tf_train_labels, logits = logits)
    )
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases
    )
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
    

In [15]:
num_steps = 3001

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomied.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict = feed_dict
        )
        
        if step % 500 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    print("Test Accuracy : {}".format(accuracy(test_prediction.eval(), test_labels)))
        
        

Initialized
Minibatch loss at step 0: 18.552256
Minibatch accuracy: 9.4%
Validation accuracy: 12.3%
Minibatch loss at step 500: 1.103186
Minibatch accuracy: 78.1%
Validation accuracy: 75.1%
Minibatch loss at step 1000: 1.439963
Minibatch accuracy: 69.5%
Validation accuracy: 75.8%
Minibatch loss at step 1500: 1.089472
Minibatch accuracy: 79.7%
Validation accuracy: 76.8%
Minibatch loss at step 2000: 1.248759
Minibatch accuracy: 69.5%
Validation accuracy: 77.7%
Minibatch loss at step 2500: 1.012444
Minibatch accuracy: 76.6%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 1.355681
Minibatch accuracy: 74.2%
Validation accuracy: 78.4%
Test accuracy: 86.0%
Test Accuracy : 85.96


### Problem
<hr />
Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units nn.relu() and 1024 hidden nodes. This model should improve your validation / test accuracy.



In [19]:
hidden_nodes = 1024
batch_size = 128

def computation(dataset, weights, biases):
    weight_sum = tf.add(tf.matmul(dataset, weights[0]), biases[0])
    hidden_layer = tf.nn.relu(weight_sum)
    logits = tf.add(tf.matmul(hidden_layer, weights[1]), biases[1])
    return logits

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed 
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labes))
    
    lmbda = tf.placeholder(tf.float32) # add placeholder
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights = [
        tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes])),
        tf.Variable(tf.truncated_normal([hidden_nodes, num_labes]))
    ]
    biases = [tf.Variable(tf.zeros([hidden_nodes])), tf.Variable(tf.zeros([num_labes]))]
    
    # Training computation.
    # logits = tf.matmul(tf_train_dataset, weights) + biases
    # loss = tf.reduce_mean(
    #     tf.nn.softmax_cross_entropy_with_logits_v2(labels = tf_train_labels, logits = logits)
    # )
    
    logits = computation(tf_train_dataset, weights, biases)
    loss = tf.add(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = tf_train_labels, logits = logits)) 
    , lmbda * (tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1]))) # add regularization
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(computation(tf_valid_dataset, weights, biases))
    test_prediction = tf.nn.softmax(computation(tf_test_dataset, weights, biases))
    
num_steps = 9001

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomied.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, lmbda: 1e-3}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict = feed_dict
        )
        
        if step % 500 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    print("Test Accuracy : {}".format(accuracy(test_prediction.eval(), test_labels)))
        
        


Initialized
Minibatch loss at step 0: 628.888550
Minibatch accuracy: 7.8%
Validation accuracy: 29.5%
Minibatch loss at step 500: 198.917664
Minibatch accuracy: 81.2%
Validation accuracy: 77.3%
Minibatch loss at step 1000: 114.181610
Minibatch accuracy: 80.5%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 68.298615
Minibatch accuracy: 85.2%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 41.666100
Minibatch accuracy: 82.0%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.374466
Minibatch accuracy: 85.2%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 15.539064
Minibatch accuracy: 85.2%
Validation accuracy: 86.4%
Minibatch loss at step 3500: 9.594486
Minibatch accuracy: 86.7%
Validation accuracy: 87.3%
Minibatch loss at step 4000: 5.869603
Minibatch accuracy: 92.2%
Validation accuracy: 87.5%
Minibatch loss at step 4500: 3.790650
Minibatch accuracy: 90.6%
Validation accuracy: 87.8%
Minibatch loss at step 5000: 2.553597
Minibatch accuracy: 92.2%
Validatio